# 신경망의 톱니바퀴 : 텐서 연산

심층 신경망이 학습한 모든 변환을 수치 데이터 텐서에 적용하는 몇 종류의 텐서 연산으로 나타낼 수 있습니다. 예를 들어 텐서 덧셈이나 텐서 곱셈 등<br>
케라스의 층은 다음과 같이 생성합니다.

kearas.layers.Dense(512, activation='relu')

이 층은 2D 텐서를 입력으로 받고 또 다른 2D 텐서를 반환하는 함수처럼 해석할 수 있습니다.

구체적으로 보면 이 함수는 다음과 같습니다.

output = relu(dot(W, input)+b)

<h3>3개의 텐서 연산</h3>

- 입력 텐서와 텐서 W 사이의 점곱(dot)
- 점곱의 결과인 2D 텐서와 벡터 b 사이의 덧셈
- relu 연산

relu()함수는 입력이 0보다 크면 입력을 그대로 반환하고 0보다 작으면 0을 반환합니다.

## 원소별 연산
relu 함수와 덧셈은 원소별 연산이며, 이 연산은 텐서에 있는 각 원소에 독립적으로 적용됩니다.

파이썬으로 단순한 원소별 연산을 구현한다면 relu 연산 구현처럼 for 반벅문을 사용할 것입니다.

In [1]:
def naive_relu(x):
    assert len(x.shape) == 2 # x는 2D 넘파이 배열입니다.
    
    x=x.copy() # 입력 텐서 자체를 바꾸지 않도록 복사합니다.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j] = max(x[i,j],0)
    return x

In [2]:
# 덧셈도 동일합니다.
def naive_add(x,y):
    assert len(x.shape) == 2 # x와 y는 2D 넘파이 배열입니다.
    assert x.shape == y.shape
    
    x=x.copy() # 입력 텐서 자체를 바꾸지 않도록 복사합니다.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j] += y[i,j]
    return x

넘파이는 시스템에 설치된 BLAS 구현에 복잡한 일들을 위임하여 다음과 같은 원소별 연산을 빠르게 처리합니다.

In [5]:
import numpy as np
x = 1
y = 2
z = x + y # 원소별 덧셈
z = np.maximum(z, 0.) # 원소별 렐루 함수

## 브로드 캐스팅
앞에서 구현한 naive_add는 동일한 크기의 2D 텐서만 지원합니다. 하지만 이전에 보았던 Dense층에서는 2D 텐서와 벡터를 더했습니다.<br>
크기가 다른 두 텐서가 더해진다면 작은 텐서가 큰 텐서의 크기에 맞추어 <strong>브로드캐스팅</strong> 됩니다.

브로드캐스팅은 두 단계로 이루어집니다.

1. 큰 텐서의 ndim에 맞도록 작은 텐서에 축(브로드캐스팅 축)이 추가됩니다.
2. 작은 텐서가 새 축을 따라서 큰 텐서의 크기에 맞도록 반복됩니다.

x의 크기가 (32, 10)이고 y의 크기가 (10,)이라면 y에 비어있는 첫번째 축을 추가하여 (1,10)으로 변경하고 32번 반복하여 (32,10)의 크기가 됩니다.<br>
여기에서 y[i, :] == y for i in range(0, 32) 입니다.

In [7]:
# 단순하게 구현한 예입니다.
def naive_add_matrix_and_vector(x,y):
    assert len(x.shape) == 2 # x는 2D 넘파이 배열입니다.
    assert len(y.shape) == 1 # y는 넘파이 벡터입니다.
    assert x.shape[1] == y.shape[0]
    
    x = x.copy() # 입력 텐서를 바꾸지 않도록 복사합니다.
    for i in range(x.shape[0]):
        for j in range(x.shpae[1]):
            x[i,j] += y[j]
    
    return x

In [10]:
# 크기가 다른 두 텐서에 브로드캐스팅으로 원소별 maximum 연산을 적용하는 예
x = np.random.random((64,3,32,10))# x는 (64,3,32,10) 크기의 랜덤 텐서
y = np.random.random((32,10))# y는 (32,10) 크기의 랜덤 텐서

z = np.maximum(x, y)# 출력 z 크기는 x와 동일하게 (64, 3, 32, 10)입니다.

In [12]:
z.ndim

4

## 텐서 점곱
<strong>텐서 곱셈</strong>이라고도 부르는 점곱 연산은 가장 널리 사용되는 유용한 텐서 연산입니다. 원소별 연산과 반대로 입력 텐서의 원소들을 결합시킵니다.

넘파이와 케라스에서는 점곱 연산에 보편적인 dot 연산자를 사용합니다.

In [38]:
x=np.array([1,2,3,4])
y=np.array([5,6,7,8])
z = np.dot(x,y) # z = x*y; sum(z) 와 동일하다.
z

70

In [39]:
# 두개의 벡터 x와 y의 점곱 계산
def naive_vector_dot(x,y):
    # x와 y는 넘파이 벡터입니다.
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    
    z=0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z

In [42]:
naive_vector_dot(x,y)

70.0

두 벡터의 점곱은 스칼라가 되므로 원소 개수가 같은 벡터끼리 점곱이 가능합니다.

In [64]:
# 행렬과 벡터 사이에서의 점곱
def naive_matrix_vector_dot(x,y):
    assert len(x.shape) == 2 # x는 넘파이 행렬입니다.
    assert len(y.shape) == 1 # y는 넘파이 벡터입니다.
    assert x.shape[1] == y.shape[0]# x의 두 번째 차원이 y의 첫 번째 차원과 같아야 합니다.
    
    z = np.zeros(x.shape[0])# x의 행과 같은 크기의 0이 채워진 벡터를 만듭니다.
    for i in range(x.shape[0]):
#         for j in range(x.shape[1]):
#             z[i] += x[i,j] * y[j]
        z[i] = naive_vector_dot(x[i,:],y)
    return z

In [65]:
x = np.array([[1,2,3,4,5],[6,7,8,9,10]])
y = np.array([5,4,3,2,1])

In [73]:
x.ndim

2

In [74]:
y.ndim

1

In [68]:
z = naive_matrix_vector_dot(x,y)
z

array([ 35., 110.])

## 행렬간의 점곱
x.shape[1] == y.shape[0] 일 때 두 행렬 x와 y의 점곱(dot(x, y))이 성립됩니다. x의 행과 y의 열 사이 벡터 점곱으로 인해 (x.shape[0], y.shape[1])크기의 행렬이 됩니다.

In [75]:
def naive_matrix_dot(x,y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]# x의 두 번째 찬원이 y의 두 번째 차원과 같아야 합니다.
    
    z = np.zeros((x.shape[0], y.shape[1]))# 0이 채워진 특정 크기의 벡터를 만듭니다.
    for i in range(x.shape[0]):# x행을 반복합니다.
        for j in range(y.shape[1]): #y열을 반복합니다.
            row_x = x[i,:]
            col_y = y[:,j]
            z[i,j] = naive_vector_dot(row_x,col_y)
    return z

In [88]:
x = np.random.randint(1,10,size=(5,10))
x

array([[6, 1, 5, 1, 9, 9, 7, 9, 3, 3],
       [5, 5, 8, 7, 2, 9, 2, 5, 8, 9],
       [2, 8, 9, 6, 6, 6, 2, 8, 8, 2],
       [5, 1, 4, 9, 7, 1, 2, 6, 7, 2],
       [5, 9, 8, 9, 6, 5, 3, 6, 6, 2]])

In [89]:
x.shape

(5, 10)

In [98]:
y = np.random.randint(1,10,size=(10,3))
y

array([[7, 2, 2],
       [8, 7, 4],
       [1, 3, 9],
       [2, 2, 9],
       [3, 7, 6],
       [9, 1, 4],
       [7, 4, 3],
       [5, 4, 5],
       [8, 1, 7],
       [1, 4, 4]])

In [99]:
y.shape

(10, 3)

In [100]:
naive_matrix_dot(x,y)

array([[286., 187., 259.],
       [296., 178., 336.],
       [291., 203., 341.],
       [197., 144., 270.],
       [297., 212., 344.]])

## 텐서 크기의 변환
숫자 데이터를 전처리할 때 사용했습니다.

train_images = train_images.reshape((60000,28*28))

텐서의 크기를 변환한다는 것은 특정 크기에 맞게 열과 행을 재배열한다는 뜻이며 크기가 변환된 텐서는 원래 텐서와 원소 개수가 동일합니다.

In [101]:
x = np.array([[0,1],[2,3],[4,5]])
x.shape

(3, 2)

In [103]:
x = x.reshape(6,1)
x

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5]])

In [104]:
x = x.reshape(2,3)
x

array([[0, 1, 2],
       [3, 4, 5]])

<strong>전치 : </strong>행과 열을 바꾸는 것

In [106]:
x = np.zeros((300,20))# 모두 0으로 채워진 (300, 20) 크기의 행렬을 만듭니다.
print(x.shape)
x = np.transpose(x)
print(x.shape)

(300, 20)
(20, 300)


# 신경망의 엔진 : 그래디언트 기반 최적화
훈련 반복 루프는 다음과 같은 역전파 알고리즘을 통해 반복되고 훈련됩니다.

1. 훈련 샘플 x와 상응하는 타깃 y의 배치를 추출합니다.
2. x를 사용하여 네트워크를 실행하고, 예측값을 구합니다.
3. 실제값과 예측값의 차이로 현재 배치의 네트워크에 대한 손실을 계산합니다.
4. 배치에 대한 손실이 감소하도록 네트워크의 가중치를 업데이트합니다.

이 과정에서 가중치를 업데이트하기 위한 효율적인 방법으로 미분 가능한 점을 활용하여, 그래디언트를 계산하여 업데이트 하는 것이 더 좋은 방법입니다.<br>
그래디언트의 반대 방향으로 가중치를 이동하면 손실이 감소합니다.

## 변화율이란?
f(x) = y 함수가 연속적이라면 x를 바꾸면 y가 변경될 것입니다. x를 epsilon_x 만큼 증가시켰을 때 y가 epsilon_y만큼 바뀐다고 할수 있습니다.

f(x+epsilon_x) = y + epsilon_y

epsilon_x가 충분히 작다면 어떤 포인트 p에서 기울기 a의 선형 함수로 f를 근사할 수 있습니다. 따라서 epsilon_y == a * epsilon_x가 됩니다.

f(x+epsilon_x) = y + a * epsilon_x

f(x)를 최소화하기 위해 epsilon_x만큼 x를 업데이트하고 싶을 때 f의 변화율을 알고 있으면 해결 됩니다. 변화율 함수는 x가 바뀜에 따라 f(x)가 어떻게 바뀔지 설명해 줍니다. f(x)의 값을 감소 시키고 싶다면 x를 변화율의 방향과 반대로 이동해야 합니다.

즉, f(x)(미분 계수)가 양수(상향선)라면 x를 줄여야하고, f(x)(미분 계수)가 음수(하향선)라면 x를 늘려야한다.

[p.s : https://cding.tistory.com/56 ]

## 텐서 연산의 변화율 : 그래디언트
위의 변화율을 텐서 연산에 적용한 것이 그래디언트입니다. 다차원 입력인 텐서를 입력 받는 함수에 변화율 개념을 확장시킨 것 입니다.<br>
값에 따라 너무 크게 변화하지 않게 하기위하여 스케일링 비율인 step값을 잘 설정해야합니다.

W1 = W0 - step * gredient(f)(W0)

이 말은 기울기가 작아지는 곡면의 낮은 위치로 이동된다는 의미입니다.

## 확률적 경사 하강법
변화율이 0이 되는 지점을 모두 찾고 이 중에서 어떤 포인트의 함수값이 가장 자은지 확인하는 것입니다.

1. 훈련 샘플 배치 x와 이에 상응하는 타깃 y를 추출합니다.
2. x로 네트워크를 실행하고 예측값을 구합니다.
3. 예측값과 실제값 사이의 오차를 측정하여 네트워크의 손실을 계산합니다.
4. 네트워크의 파라미터에 대한 손실 함수의 그래디언트를 계산합니다(backward pass)
5. 그래디언트의 반대 방향으로 파라미터를 조금 이동시킵니다. W -= step * gredient

## 변화율 연결 : 역전파 알고리즘
연쇄 법칙(chain rule)이라 불리는 f(g(x))′=f′g(x)∗g′(x) 를 이용하여 많은 층의 값들을 경사 하강법으로 연산할 수 있습니다.

현재는 텐서플로처럼 기호 미분이 가능한 최신 프레임워크를 활용하여 미분값을 구하니, 역전파 알고리즘을 직접 구현하고 역전파 공식을 유도하는 등의 시간 및 노력의 소모가 필요 없습니다. 

## 